# Imports and Setting Up

In [ ]:
import json
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

### TODO: Add filename

ncd_file = ''

In [ ]:
data = pd.read_json(ncd_file, lines=True)

# Web Scraping Functions

In [ ]:
def scrape_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article_text = ''
        for paragraph in soup.find_all('div', class_='primary-cli cli cli-text'):
            article_text += paragraph.get_text() + '\n'
        return article_text.strip()
    except Exception as e:
        print("Error occurred while scraping:", e)
        return None

In [ ]:
executor = ThreadPoolExecutor(max_workers=10)

In [ ]:
def scrape_single_url(row):
    url = row.link
    try:
        article_text = scrape_article_text(url)
        return {'link': url, 'article_text': article_text}
    except Exception as e:
        print("Error occurred while scraping:", e)
        return {'link': url, 'article_text': None}

In [ ]:
subset_data = data.iloc[:11842]
subset_data['article_text'] = subset_data['link'].apply(scrape_article_text)

results = executor.map(scrape_single_url, subset_data.itertuples())

# Convert the results to a list of dictionaries and create new df
results_list = list(results)
result_df = pd.DataFrame(results_list)

# Merge result with original
df = pd.merge(subset_data, result_df, on='link', how='left')

In [ ]:
results = executor.map(scrape_single_url, data.itertuples())
results_list = list(results)
result_df = pd.DataFrame(results_list)

In [ ]:
df = df[['link', 'headline', 'category', 'short_description', 'authors', 'date', 'article_text_x']]
classification_df = df
classification_df['class'] = df['category'].map(lambda x: 1 if x == 'POLITICS' else 0)
classification_df['text'] = classification_df['headline'] + classification_df['short_description'] + classification_df['article_text_x']
## TODO: Change filepath
classification_df.to_csv('/filepath/classifierdata_raw.csv', index=False)